# Importar Librerías

In [1]:
import datetime
import pickle
import polars as pl
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar Datos

In [2]:
cols_df1 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOUNITARIO',
    'CANTIDAD',
    'MONTORECLAMADO',
    'MONTOAJUSTADO',
    'MONTODIFERENCIAAFILIADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'TipoInstitucionPrestador'
    ]

cols_df2 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOAJUSTADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'MONTOEXCEPCION',
    'MONTOCONCESION',
    'MONTOAUTORIZADO'
    ]


In [3]:
# Data Autorizaciones

df = pl.read_parquet(
    '../Datos/dataAutorizaciones20221208.parquet',
    columns = cols_df1
)

df = df.sample(500000)
df = df.to_pandas()

print(type(df))
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
(500000, 19)


In [4]:
# Data Distribuciones Autorizaciones

df2 = pl.read_parquet(
    '../Datos/dataDistribucionesAutorizaciones20221208.parquet',
    columns = cols_df2
)

df2 = df2.sample(500000)
df2 = df2.to_pandas()

print(type(df2))
print(df2.shape)


<class 'pandas.core.frame.DataFrame'>
(500000, 17)


In [6]:
# CUPS frecuentes en emergencias
cups_df = pd.read_excel(
    '../Datos/CUPS frecuencias y otros (002).xlsx',engine='openpyxl'
)

# CUPS cantidades esperadas
valores_referencia_cups_df = pd.read_excel(
    '../Datos/CUPS frecuencias y otros (002).xlsx', 
    sheet_name=1,engine='openpyxl'
)

# Indicador de dias de habitación

# Read File
salas_df = pd.read_excel(
    '../Datos/salas.xlsx', engine='openpyxl'
)

salas_list = salas_df['DESCRIPCION'].tolist()

/Users/jhon.pelaez/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Unir Datasets

In [7]:
# Extraer autorizaciones en los dos datasets
autorizaciones_1 = set(df.AUTORIZACION.unique())
autoriaciones_2 = set(df2.AUTORIZACION.unique())
autorizaciones = autorizaciones_1.intersection(autoriaciones_2)

tmp_df_columns = [
    'AUTORIZACION', 
    'CLIENTEAFILIADOBENEFICIARIO',
    'PRESTACION', 
    'PRESTADOR', 
    'FECHASERVICIO', 
    'MONTORECLAMADO'
]

tmp_df2_columns = [
    'AUTORIZACION', 
    'CLIENTEAFILIADOBENEFICIARIO', 
    'PRESTACION', 
    'PRESTADOR',
    'FECHASERVICIO', 
    'MONTOASEGURADORA', 
    'MONTOEXCEPCION', 
    'MONTOCONCESION', 
    'MONTOAUTORIZADO'
]

tmp_df = df[df.AUTORIZACION.isin(autorizaciones)][tmp_df_columns]
tmp_df2 = df2[df2.AUTORIZACION.isin(autorizaciones)][tmp_df2_columns]

# agrupar montos en el dataset de distribución
tmp_df2_new = tmp_df2.groupby(['AUTORIZACION', 'CLIENTEAFILIADOBENEFICIARIO', 'PRESTACION', 'PRESTADOR', 
                               'FECHASERVICIO'])[['MONTOASEGURADORA', 'MONTOEXCEPCION', 'MONTOCONCESION', 'MONTOAUTORIZADO']].sum().reset_index()

# Consolidar las dos bases de datos
consolidated_df = df.merge(
    tmp_df2_new,
    on=['AUTORIZACION', 'CLIENTEAFILIADOBENEFICIARIO','PRESTACION', 'PRESTADOR', 'FECHASERVICIO'],
)

consolidated_df.drop(columns=['MONTOASEGURADORA_x'], inplace=True)
consolidated_df.rename(
    columns={
        'MONTOASEGURADORA_y': 'MONTOASEGURADORA',
        'MONTODIFERENCIAAFILIADO': 'MONTOUSUARIO',
        'TipoPrestador': 'TIPOPRESTADOR',
    },
    inplace=True
)

In [8]:
del(df, df2, tmp_df, tmp_df2, tmp_df2_new)

In [9]:
consolidated_df.shape

(16521, 22)